In [1]:
import pandas as pd
import numpy as np

In [2]:
school_csv = "../Resources/schools_complete.csv"
student_csv = "../Resources/students_complete.csv"

In [3]:
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

In [4]:
school_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [5]:
student_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [6]:
district_df = pd.merge(school_df,student_df, how="left", on=["school_name","school_name"])
district_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [7]:
school_name = district_df["school_name"].unique()
school_name

array(['Huang High School', 'Figueroa High School', 'Shelton High School',
       'Hernandez High School', 'Griffin High School',
       'Wilson High School', 'Cabrera High School', 'Bailey High School',
       'Holden High School', 'Pena High School', 'Wright High School',
       'Rodriguez High School', 'Johnson High School', 'Ford High School',
       'Thomas High School'], dtype=object)

In [8]:
total_schools = len(school_name)
total_schools

15

In [9]:
total_students = district_df["Student ID"].count()
total_students

39170

In [10]:
total_budget = school_df["budget"].sum()
total_budget

24649428

In [11]:
district_avg_math = round(district_df["math_score"].mean(),2)
district_avg_math

78.99

In [12]:
district_avg_read = round(district_df["reading_score"].mean(),2)
district_avg_read

81.88

In [13]:
dpass_math = district_df[district_df.math_score >= 70]
dperc_math = round((dpass_math.student_name.count()/total_students)*100,2)
dperc_math

74.98

In [14]:
dpass_read = district_df[district_df.reading_score >= 70]
dperc_read = round((dpass_read.student_name.count()/total_students)*100,2)
dperc_read

85.81

In [15]:
doverall_pass = round((district_avg_math + district_avg_read)/2,2)
doverall_pass

80.44

In [16]:
newDistrict_df = pd.DataFrame ({
    "Total Schools" : [total_schools],
    "Total Students" : [total_students],
    "Total Budget" : [total_budget],
    "Avg. Math Score" : [district_avg_math],
    "Avg. Reading Score" : [district_avg_read],
    "% Passing Math" : [dperc_math],
    "% Passing Reading" : [dperc_read],
    "%Overall Pass Rate" : [doverall_pass]})

newDistrict_df["Total Budget"] = newDistrict_df["Total Budget"].apply(lambda x: "${:.1f}k".format((x/1000)))
newDistrict_df["Total Students"] = newDistrict_df["Total Students"].apply(lambda x:"{:,.0f}".format(x))
newDistrict_df.head()

,Total Schools,Total Students,Total Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,%Overall Pass Rate
0,15,"39,170",$24649.4k,78.99,81.88,74.98,85.81,80.44


In [17]:
school_df=district_df.rename(columns={"school_name": "School Name"})

school_df.loc[school_df.math_score >= 70,'MathPass'] = 1
school_df.loc[school_df.reading_score >= 70,'ReadPass'] = 1

df2 = school_df.groupby('School Name').agg({
       'type':"first",
       'School Name':"count",
       'budget':"first",
       'math_score': "mean",
       'reading_score':"mean",
       'MathPass':'sum',
       'ReadPass':'sum'
   })

df3 = df2.rename(columns={"type": "School Type",
                         "School Name": "Total Students",
                         "budget":"Total School Budget",
                         "math_score": "Avg Math Score",
                         "reading_score": "Avg Reading Score"})

df3['Per Student Budget'] = (df3['Total School Budget']/df3['Total Students'])
df3['% Passing Math'] = round((df3['MathPass']/df3['Total Students'])*100,2)
df3['% Passing Reading'] = round((df3['ReadPass']/df3['Total Students'])*100,2)
df3['Overall % Passing'] = round((df3['% Passing Math']+df3['% Passing Reading'])/2,2)

df4 = df3.drop(['MathPass','ReadPass'], axis = 1)

df4['Per Student Budget'] = df4['Per Student Budget'].apply(lambda x:"${:,.2f}".format(x))
df4['Total School Budget'] = df4['Total School Budget'].apply(lambda x:"${:,.2f}".format(x))
df4['Avg Math Score'] = df4['Avg Math Score'].apply(lambda x:"{:,.2f}".format(x))
df4['Avg Reading Score'] = df4['Avg Reading Score'].apply(lambda x:"{:,.2f}".format(x))
df4['Total Students'] = df4['Total Students'].apply(lambda x:"{:,.0f}".format(x))
df4.head(15)

,School Type,Total Students,Total School Budget,Avg Math Score,Avg Reading Score,Per Student Budget,% Passing Math,% Passing Reading,Overall % Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",77.05,81.03,$628.00,66.68,81.93,74.31
Cabrera High School,Charter,"1,858","$1,081,356.00",83.06,83.98,$582.00,94.13,97.04,95.58
Figueroa High School,District,"2,949","$1,884,411.00",76.71,81.16,$639.00,65.99,80.74,73.36
Ford High School,District,"2,739","$1,763,916.00",77.10,80.75,$644.00,68.31,79.30,73.81
Griffin High School,Charter,"1,468","$917,500.00",83.35,83.82,$625.00,93.39,97.14,95.26
Hernandez High School,District,"4,635","$3,022,020.00",77.29,80.93,$652.00,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087.00",83.80,83.81,$581.00,92.51,96.25,94.38
Huang High School,District,"2,917","$1,910,635.00",76.63,81.18,$655.00,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650.00",77.07,80.97,$650.00,66.06,81.22,73.64


In [18]:
top_performing_schools = df4.nlargest(5, "Overall % Passing")
top_performing_schools

,School Type,Total Students,Total School Budget,Avg Math Score,Avg Reading Score,Per Student Budget,% Passing Math,% Passing Reading,Overall % Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",83.06,83.98,$582.00,94.13,97.04,95.58
Thomas High School,Charter,"1,635","$1,043,130.00",83.42,83.85,$638.00,93.27,97.31,95.29
Pena High School,Charter,962,"$585,858.00",83.84,84.04,$609.00,94.59,95.95,95.27
Griffin High School,Charter,"1,468","$917,500.00",83.35,83.82,$625.00,93.39,97.14,95.26
Wilson High School,Charter,"2,283","$1,319,574.00",83.27,83.99,$578.00,93.87,96.54,95.21


In [19]:
bottom_performing_schools = df4.nsmallest(5, "Overall % Passing")
bottom_performing_schools

,School Type,Total Students,Total School Budget,Avg Math Score,Avg Reading Score,Per Student Budget,% Passing Math,% Passing Reading,Overall % Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",76.84,80.74,$637.00,66.37,80.22,73.30
Figueroa High School,District,"2,949","$1,884,411.00",76.71,81.16,$639.00,65.99,80.74,73.36
Huang High School,District,"2,917","$1,910,635.00",76.63,81.18,$655.00,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650.00",77.07,80.97,$650.00,66.06,81.22,73.64
Ford High School,District,"2,739","$1,763,916.00",77.10,80.75,$644.00,68.31,79.30,73.81


In [20]:
mathbygrade = district_df.groupby(['school_name','grade']).agg({
       'math_score': "mean"
   })
mathbygrade

math_score
school_name           grade            
Bailey High School    10th    76.996772
                      11th    77.515588
                      12th    76.492218
                      9th     77.083676
Cabrera High School   10th    83.154506
                      11th    82.765560
                      12th    83.277487
                      9th     83.094697
Figueroa High School  10th    76.539974
                      11th    76.884344
                      12th    77.151369
                      9th     76.403037
Ford High School      10th    77.672316
                      11th    76.918058
                      12th    76.179963
                      9th     77.361345
Griffin High School   10th    84.229064
                      11th    83.842105
                      12th    83.356164
                      9th     82.044010
Hernandez High School 10th    77.337408
                      11th    77.136029
                      12th    77.186567
                      9th     77.438495
Holden High School    10th    83.429825
                      11th    85.000000
                      12th    82.855422
                      9th     83.787402
Huang High School     10th    75.908735
                      11th    76.446602
                      12th    77.225641
                      9th     77.027251
Johnson High School   10th    76.691117
                      11th    77.491653
                      12th    76.863248
                      9th     77.187857
Pena High School      10th    83.372000
                      11th    84.328125
                      12th    84.121547
                      9th     83.625455
Rodriguez High School 10th    76.612500
                      11th    76.395626
                      12th    77.690748
                      9th     76.859966
Shelton High School   10th    82.917411
                      11th    83.383495
                      12th    83.778976
                      9th     83.420755
Thomas High School    10th    83.087886
                      11th    83.498795
                      12th    83.497041
                      9th     83.590022
Wilson High School    10th    83.724422
                      11th    83.195326
                      12th    83.035794
                      9th     83.085578
Wright High School    10th    84.010288
                      11th    83.836782
                      12th    83.644986
                      9th     83.264706

In [21]:
readingbygrade = district_df.groupby(['school_name','grade']).agg({
       'reading_score': "mean"
   })
readingbygrade

reading_score
school_name           grade               
Bailey High School    10th       80.907183
                      11th       80.945643
                      12th       80.912451
                      9th        81.303155
Cabrera High School   10th       84.253219
                      11th       83.788382
                      12th       84.287958
                      9th        83.676136
Figueroa High School  10th       81.408912
                      11th       80.640339
                      12th       81.384863
                      9th        81.198598
Ford High School      10th       81.262712
                      11th       80.403642
                      12th       80.662338
                      9th        80.632653
Griffin High School   10th       83.706897
                      11th       84.288089
                      12th       84.013699
                      9th        83.369193
Hernandez High School 10th       80.660147
                      11th       81.396140
                      12th       80.857143
                      9th        80.866860
Holden High School    10th       83.324561
                      11th       83.815534
                      12th       84.698795
                      9th        83.677165
Huang High School     10th       81.512386
                      11th       81.417476
                      12th       80.305983
                      9th        81.290284
Johnson High School   10th       80.773431
                      11th       80.616027
                      12th       81.227564
                      9th        81.260714
Pena High School      10th       83.612000
                      11th       84.335938
                      12th       84.591160
                      9th        83.807273
Rodriguez High School 10th       80.629808
                      11th       80.864811
                      12th       80.376426
                      9th        80.993127
Shelton High School   10th       83.441964
                      11th       84.373786
                      12th       82.781671
                      9th        84.122642
Thomas High School    10th       84.254157
                      11th       83.585542
                      12th       83.831361
                      9th        83.728850
Wilson High School    10th       84.021452
                      11th       83.764608
                      12th       84.317673
                      9th        83.939778
Wright High School    10th       83.812757
                      11th       84.156322
                      12th       84.073171
                      9th        83.833333

In [26]:
bins = [0, 599, 624, 650, 700]
group_names = ["599 or less", "600-624", "625-650", "651-700"]
pd.cut(df3["Per Student Budget"], bins, labels=group_names).head()

School Name
Bailey High School          625-650
Cabrera High School     599 or less
Figueroa High School        625-650
Ford High School            625-650
Griffin High School         625-650
Name: Per Student Budget, dtype: category
Categories (4, object): [599 or less < 600-624 < 625-650 < 651-700]

In [28]:
df3["Budget Group"] = pd.cut(df3["Per Student Budget"], bins, labels=group_names)
df3.head(15)

,School Type,Total Students,Total School Budget,Avg Math Score,Avg Reading Score,MathPass,ReadPass,Per Student Budget,% Passing Math,% Passing Reading,Overall % Passing,Budget Group
School Name,,,,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,3318.0,4077.0,628.0,66.68,81.93,74.31,625-650
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1749.0,1803.0,582.0,94.13,97.04,95.58,599 or less
Figueroa High School,District,2949,1884411,76.711767,81.158020,1946.0,2381.0,639.0,65.99,80.74,73.36,625-650
Ford High School,District,2739,1763916,77.102592,80.746258,1871.0,2172.0,644.0,68.31,79.30,73.81,625-650
Griffin High School,Charter,1468,917500,83.351499,83.816757,1371.0,1426.0,625.0,93.39,97.14,95.26,625-650
Hernandez High School,District,4635,3022020,77.289752,80.934412,3094.0,3748.0,652.0,66.75,80.86,73.81,651-700
Holden High School,Charter,427,248087,83.803279,83.814988,395.0,411.0,581.0,92.51,96.25,94.38,599 or less
Huang High School,District,2917,1910635,76.629414,81.182722,1916.0,2372.0,655.0,65.68,81.32,73.50,651-700
Johnson High School,District,4761,3094650,77.072464,80.966394,3145.0,3867.0,650.0,66.06,81.22,73.64,625-650


In [31]:
bins = [0, 1000, 2500, 5000]
group_names = ["Small", "Medium", "Large"]
pd.cut(df3["Total Students"], bins, labels=group_names).head()

School Name
Bailey High School       Large
Cabrera High School     Medium
Figueroa High School     Large
Ford High School         Large
Griffin High School     Medium
Name: Total Students, dtype: category
Categories (3, object): [Small < Medium < Large]

In [32]:
df3["School Size"] = pd.cut(df3["Total Students"], bins, labels=group_names)
df3.head(15)

,School Type,Total Students,Total School Budget,Avg Math Score,Avg Reading Score,MathPass,ReadPass,Per Student Budget,% Passing Math,% Passing Reading,Overall % Passing,Budget Group,School Size
School Name,,,,,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,3318.0,4077.0,628.0,66.68,81.93,74.31,625-650,Large
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1749.0,1803.0,582.0,94.13,97.04,95.58,599 or less,Medium
Figueroa High School,District,2949,1884411,76.711767,81.158020,1946.0,2381.0,639.0,65.99,80.74,73.36,625-650,Large
Ford High School,District,2739,1763916,77.102592,80.746258,1871.0,2172.0,644.0,68.31,79.30,73.81,625-650,Large
Griffin High School,Charter,1468,917500,83.351499,83.816757,1371.0,1426.0,625.0,93.39,97.14,95.26,625-650,Medium
Hernandez High School,District,4635,3022020,77.289752,80.934412,3094.0,3748.0,652.0,66.75,80.86,73.81,651-700,Large
Holden High School,Charter,427,248087,83.803279,83.814988,395.0,411.0,581.0,92.51,96.25,94.38,599 or less,Small
Huang High School,District,2917,1910635,76.629414,81.182722,1916.0,2372.0,655.0,65.68,81.32,73.50,651-700,Large
Johnson High School,District,4761,3094650,77.072464,80.966394,3145.0,3867.0,650.0,66.06,81.22,73.64,625-650,Large


In [35]:
df3.sort_values(["School Name","School Type"])
df3

,School Type,Total Students,Total School Budget,Avg Math Score,Avg Reading Score,MathPass,ReadPass,Per Student Budget,% Passing Math,% Passing Reading,Overall % Passing,Budget Group,School Size
School Name,,,,,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,3318.0,4077.0,628.0,66.68,81.93,74.31,625-650,Large
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,1749.0,1803.0,582.0,94.13,97.04,95.58,599 or less,Medium
Figueroa High School,District,2949,1884411,76.711767,81.158020,1946.0,2381.0,639.0,65.99,80.74,73.36,625-650,Large
Ford High School,District,2739,1763916,77.102592,80.746258,1871.0,2172.0,644.0,68.31,79.30,73.81,625-650,Large
Griffin High School,Charter,1468,917500,83.351499,83.816757,1371.0,1426.0,625.0,93.39,97.14,95.26,625-650,Medium
Hernandez High School,District,4635,3022020,77.289752,80.934412,3094.0,3748.0,652.0,66.75,80.86,73.81,651-700,Large
Holden High School,Charter,427,248087,83.803279,83.814988,395.0,411.0,581.0,92.51,96.25,94.38,599 or less,Small
Huang High School,District,2917,1910635,76.629414,81.182722,1916.0,2372.0,655.0,65.68,81.32,73.50,651-700,Large
Johnson High School,District,4761,3094650,77.072464,80.966394,3145.0,3867.0,650.0,66.06,81.22,73.64,625-650,Large
